In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd
from skimage import io, transform
import PIL
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./image_files', train=True,
                                        download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16,
                                          shuffle=True, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


In [3]:
def imshow(img):
    unnormalized_image = img / 2 + 0.5     # unnormalize
    npimg = unnormalized_image.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(3, 32, 5, padding=(2,2))
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=(1,1))
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=(1,1))
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=(1,1))
        self.bn4 = nn.BatchNorm2d(64)
        self.conv5 = nn.Conv2d(64, 64, 3, padding=(1,1))
        self.bn5 = nn.BatchNorm2d(64)
        self.conv6 = nn.Conv2d(64, 128, 3, padding=(1,1))
        self.bn6 = nn.BatchNorm2d(128)
        self.conv7 = nn.Conv2d(128, 128, 3, padding=(1,1))
        self.bn7 = nn.BatchNorm2d(128)
        self.conv8 = nn.Conv2d(128, 128, 3, padding=(1,1))
        self.bn8 = nn.BatchNorm2d(128)
        self.conv9 = nn.Conv2d(128, 128, 3, padding=(1,1))
        self.bn9 = nn.BatchNorm2d(128)
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(x + self.conv3(x)))
        x = F.relu(self.bn4(x + self.conv4(x)))
        x = self.pool(F.relu(self.bn5(x + self.conv5(x))))
        x = F.relu(self.bn6(self.conv6(x)))
        x = F.relu(self.bn7(x + self.conv7(x)))
        x = F.relu(self.bn8(x + self.conv8(x)))
        x = self.pool(F.relu(self.bn9(x + self.conv9(x))))
        x = x.view(-1, 128 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(self.dropout(x))
        return x


In [26]:
net = Net().cuda()
criterion = nn.CrossEntropyLoss()


In [27]:
for epoch in range(10):  # loop over the dataset multiple times
    optimizer = optim.SGD(net.parameters(), lr=5e-3 * (1.25 ** (-epoch)), momentum=0.9)
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs_cpu, labels_cpu = data
        inputs, labels = inputs_cpu.cuda(), labels_cpu.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0


[1,   100] loss: 2.105
[1,   200] loss: 1.933
[1,   300] loss: 1.837
[1,   400] loss: 1.769
[1,   500] loss: 1.746
[1,   600] loss: 1.685
[1,   700] loss: 1.678
[1,   800] loss: 1.631
[1,   900] loss: 1.616
[1,  1000] loss: 1.581
[1,  1100] loss: 1.506
[1,  1200] loss: 1.511
[1,  1300] loss: 1.567
[1,  1400] loss: 1.476
[1,  1500] loss: 1.483
[1,  1600] loss: 1.395
[1,  1700] loss: 1.448
[1,  1800] loss: 1.373
[1,  1900] loss: 1.378
[1,  2000] loss: 1.319
[1,  2100] loss: 1.383
[1,  2200] loss: 1.317
[1,  2300] loss: 1.323
[1,  2400] loss: 1.267
[1,  2500] loss: 1.338
[1,  2600] loss: 1.224
[1,  2700] loss: 1.266
[1,  2800] loss: 1.232
[1,  2900] loss: 1.232
[1,  3000] loss: 1.192
[1,  3100] loss: 1.177
[2,   100] loss: 1.125
[2,   200] loss: 1.091
[2,   300] loss: 1.079
[2,   400] loss: 1.068
[2,   500] loss: 1.099
[2,   600] loss: 1.063
[2,   700] loss: 1.078
[2,   800] loss: 1.103
[2,   900] loss: 1.071
[2,  1000] loss: 1.016
[2,  1100] loss: 1.022
[2,  1200] loss: 1.071
[2,  1300] 

In [28]:
testset = torchvision.datasets.CIFAR10(root='./image_files', train=False,
                                       download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)


In [29]:
net.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images_cpu, labels_cpu = data
        images, labels = images_cpu.cuda(), labels_cpu.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 81 %


In [30]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images_cpu, labels_cpu = data
        images, labels = images_cpu.cuda(), labels_cpu.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))


Accuracy of plane : 86 %
Accuracy of   car : 96 %
Accuracy of  bird : 78 %
Accuracy of   cat : 61 %
Accuracy of  deer : 74 %
Accuracy of   dog : 72 %
Accuracy of  frog : 83 %
Accuracy of horse : 88 %
Accuracy of  ship : 90 %
Accuracy of truck : 79 %
